## Recursive CTE (Common Table Expression

Quellen:

- [SQL Server Recursive CTE](https:\www.sqlservertutorial.net\sql-server-basics\sql-server-recursive-cte\)
- [What Is a Recursive CTE in SQL?](https:\learnsql.com\blog\sql-recursive-cte\)
- [Recursive Queries using Common Table Expressions (CTE) in SQL Server](https:\www.mssqltips.com\sqlservertip\1520\recursive-queries-using-common-table-expressions-cte-in-sql-server\)

## Mitarbeiterhierarchie

In [ ]:
-- Manager für einen ausgewählten Mitarbeiter finden
WITH MitarbeiterCTE AS (    
    SELECT MitarbeiterNr, Vorgesetzte, Vorname, Nachname, 0 AS Distanz
    FROM Mitarbeiter
    WHERE MitarbeiterNr = 9
    UNION ALL
    SELECT m.MitarbeiterNr, m.Vorgesetzte, m.Nachname, m.Vorname, Distanz + 1
    FROM MitarbeiterCTE AS s
    JOIN Mitarbeiter AS m ON s.Vorgesetzte = m.MitarbeiterNr
)
SELECT MitarbeiterNr, Vorgesetzte, Vorname, Nachname, Distanz
FROM MitarbeiterCTE;

In [ ]:
-- Ausgehend vom Manager die untergeordnete Hierarchie finden
WITH Managers AS 
( 
--initialization 
SELECT MitarbeiterNr, Nachname, Vorgesetzte
FROM Mitarbeiter
WHERE Vorgesetzte IS NULL 
UNION ALL 
--recursive execution 
SELECT e.MitarbeiterNr,e.Nachname, e.Vorgesetzte 
FROM Mitarbeiter e INNER JOIN Managers m  
ON e.Vorgesetzte = m.MitarbeiterNr 
) 
SELECT * FROM Managers OPTION (MAXRECURSION 3); 


In [ ]:
-- Vorbereitung einer Streckensuche
-- Tabelle mit Wegstrecken zwischen Städten
USE SQL_Seminar;

DROP TABLE IF EXISTS CityRouten;
GO

CREATE TABLE CityRouten (
    city_from VARCHAR(300),
    city_to VARCHAR(300),
    distance DECIMAL(9,1),
    CONSTRAINT pk_CityRouten PRIMARY KEY(city_from, city_to)
);
GO

INSERT INTO CityRouten (city_from, city_to, distance)
VALUES ('Groningen','Heerenveen',61.4),
('Groningen','Harlingen',91.6),
('Harlingen','Wieringerwerf',52.3),
('Wieringerwerf','Hoorn',26.5),
('Hoorn','Amsterdam',46.1),
('Amsterdam','Haarlem',30),
('Heerenveen','Lelystad',74),
('Lelystad','Amsterdam',57.2);

SELECT * FROM CityRouten;




In [ ]:
-- Finde mögliche Strecken zwischen zwei Städten
WITH  possible_route AS (
   SELECT  cr.city_to,
           CAST(CONCAT(cr.city_from , ' —> ' , cr.city_to) AS VARCHAR(1000)) AS route,
           cr.distance
    FROM CityRouten cr
    WHERE cr.city_from = 'Groningen'
    UNION ALL
    SELECT  cr.city_to,
            CAST(CONCAT(pr.route, ' —> ' , cr.city_to) AS VARCHAR(1000)) AS route,
            CAST((pr.distance + cr.distance) AS DECIMAL(9, 1))
    FROM possible_route pr
    INNER JOIN CityRouten cr ON cr.city_from = pr.city_to
)
 
SELECT  pr.route, pr.distance
FROM possible_route pr
WHERE pr.city_to = 'Haarlem'
ORDER BY pr.distance;